In [ ]:
import requests #to fetch data from Blynk Cloud
import time 
from datetime import datetime #to timestamp the challan
from urllib.parse import urlencode #to create a clickable challan link
from twilio.rest import Client #to send SMS messages

# ---------- Blynk Configuration ----------
BLYNK_AUTH_TOKEN = "23apIo5Mjijir1KQu_iFTnySqh59ljkT"
PINS = ["V0", "V1", "V2"]   # V0 = Tag ID, V1 = Latitude, V2 = Longitude

# ---------- Challan Page ----------
CHALLAN_BASE = "https://kkhyathikiran.github.io/challanEcosentinel/"

# ---------- Twilio Configuration ----------c
TWILIO_ACCOUNT_SID = "000000000000000000000000000000"
TWILIO_AUTH_TOKEN = "0000000000000000000000000000000"
TWILIO_FROM_NUMBER = "000000000000000"   # Your Twilio number
TWILIO_TO_NUMBER = "+0000000000"   # Recipient’s phone number


def fetch_all_latest(auth_token, pins):
    """
    Fetch each pin individually from Blynk Cloud.
    """
    data = {}
    for pin in pins:
        url = f"https://blynk.cloud/external/api/get?token={auth_token}&{pin}"
        try:
            resp = requests.get(url, timeout=5)
            print(f"DEBUG [{pin}] → {resp.status_code} | {resp.text}")  # helpful debug
            if resp.status_code == 200 and resp.text not in ("", "null", "None"):
                data[pin] = resp.text.strip()
            else:
                data[pin] = ""
        except Exception as e:
            print(f"⚠ Error fetching {pin}: {e}")
            data[pin] = ""
    return data

# ---------- Build challan link ----------
def make_challan_link(base_url, tag_id, lat, lon, violation="throwing_dust"):
    """
    Build a challan URL like:
      https://kkhyathikiran.github.io/challanEcosentinel/
    """
    ts = datetime.utcnow().isoformat(timespec='seconds') + "Z"
    params = {
        "tag": tag_id,
        "lat": lat or "",
        "lon": lon or "",
        "violation": violation,
        "ts": ts
    }
    return base_url.rstrip("/") + "/?" + urlencode(params)

# ---------- Send SMS ----------
def send_sms(challan_link, tag_id, lat, lon):
    """
    Send challan link to the user via Twilio SMS.
    """
    try:
        client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
        body = (
            f"Violation Detected!\n"
            f"Tag ID: {tag_id}\n"
            f"Location: https://maps.google.com/?q={lat},{lon}\n"
            f"Challan Link: {challan_link}"
        )
        message = client.messages.create(
            body=body,
            from_=TWILIO_FROM_NUMBER,
            to=TWILIO_TO_NUMBER
        )
        print(f"✅ SMS sent successfully! SID: {message.sid}")
    except Exception as e:
        print(f"⚠ Failed to send SMS: {e}")

# ---------- Main Monitoring Loop ----------
if __name__ == "__main__":
    last_tag_id = None
    POLL_INTERVAL = 5  # seconds
    print("🚀 Monitoring Blynk for NFC Tag reads...")

    try:
        while True:
            data = fetch_all_latest(BLYNK_AUTH_TOKEN, PINS)
            if data:
                tag_id = data.get("V0") or data.get("v0")
                lat = data.get("V1") or data.get("v1")
                lon = data.get("V2") or data.get("v2")

                tag_id = tag_id.strip() if isinstance(tag_id, str) else tag_id
                lat = lat.strip() if isinstance(lat, str) else lat
                lon = lon.strip() if isinstance(lon, str) else lon

                if tag_id and tag_id not in ("null", "None", "") and tag_id != last_tag_id:
                    print(f"\n📍 New Tag Detected: {tag_id}")
                    print(f"Latitude: {lat}, Longitude: {lon}")

                    challan_link = make_challan_link(CHALLAN_BASE, tag_id, lat, lon)
                    print(f"🧾 Generated Challan Link: {challan_link}")

                    # Send SMS with challan link
                    send_sms(challan_link, tag_id, lat, lon)

                    last_tag_id = tag_id
                else:
                    print(".", end="", flush=True)
            else:
                print("No data received from Blynk.")
            time.sleep(POLL_INTERVAL)
    except KeyboardInterrupt:
        print("\nStopped by user.")